# Testing and Debugging in Polars

## Debugging chained operations

### How it works...

In [ ]:
import polars as pl

In [ ]:
lf = pl.scan_csv('../data/pokemon.csv')

In [ ]:
lf.fetch(5)

In [ ]:
(
    lf
    .with_columns(
        pl.col('Attack').rank(method='dense').alias('Atk Rank'),
        pl.col('Defense').rank(method='dense').alias('Def Rank'),
        pl.col('Speed').rank(method='dense').alias('Spe Rank'),
    )
    .select(
        'Name',
        'Total',
        'Attack',
        'Defense',
        'Speed',
        pl.col('^*Rank$')
    )
    .sort('Total')
    .head()
    .collect()
)

In [ ]:
(
    lf
    .with_columns(
        pl.col('Attack').rank(method='dense').alias('Atk Rank'),
        pl.col('Defense').rank(method='dense').alias('Def Rank'),
        pl.col('Speed').rank(method='dense').alias('Spe Rank'),
    )
    .select(
        'Name',
        'Total',
        'Attack',
        'Deffense',
        'Speed',
        pl.col('^*Rank$')
    )
    .sort('Total')
    .head()
    .collect()
)

In [ ]:
(
    lf
    .with_columns(
        pl.col('Attack').rank(method='dense').alias('Atk Rank'),
        pl.col('Defense').rank(method='dense').alias('Def Rank'),
        pl.col('Speed').rank(method='dense').alias('Spe Rank'),
    )
    # .select(
    #     'Name',
    #     'Total',
    #     'Attack',
    #     'Deffense',
    #     'Speed',
    #     pl.col('^*Rank$')
    # )
    .sort('Total')
    .head()
    .collect()
)

In [7]:
(
    lf
    .collect()
    .with_columns(
        pl.col('Attack').rank(method='dense').alias('Atk Rank'),
        pl.col('Defense').rank(method='dense').alias('Def Rank'),
        pl.col('Speed').rank(method='dense').alias('Spe Rank'),
    )
    .select(
        'Name',
        'Total',
        'Attack',
        'Deffense',
        'Speed',
        pl.col('^*Rank$')
    )
    .sort('Total')
    .head()
)

KeyboardInterrupt: 

In [13]:
def add_ranks(lf):
    return (
        lf
        .with_columns(
            pl.col('Attack').rank(method='dense').alias('Atk Rank'),
            pl.col('Defense').rank(method='dense').alias('Def Rank'),
            pl.col('Speed').rank(method='dense').alias('Spe Rank'),
        )
    )

def keep_cols(lf):
    return (
        lf
        .select(
            'Name',
            'Total',
            'Attack',
            'Defense',
            'Speed',
            pl.col('^*Rank$')
        )
    )

(
    lf
    .pipe(add_ranks)
    .pipe(keep_cols)
    .sort('Total')
    .head()
    .collect()
)

Name,Total,Attack,Defense,Speed,Atk Rank,Def Rank,Spe Rank
str,i64,i64,i64,i64,u32,u32,u32
"""Caterpie""",195,30,35,45,5,6,10
"""Weedle""",195,35,30,50,6,5,12
"""Magikarp""",200,10,55,80,2,16,27
"""Kakuna""",205,25,50,35,4,13,5
"""Metapod""",205,20,55,30,3,16,4
